## Grafiquitos

In [ ]:
import glob
import matplotlib.pyplot as plt
import numpy as np
import os.path
import pandas as pd
import pylab
import re
import seaborn as sns
import statsmodels
from matplotlib.colors import LogNorm

In [ ]:
BENCH_DIR = '../../bench/'
GRAPH_DIR = '../../graph/'

CSV_COLUMNS = ["gyms","stops","bagSize","time","reps","status",
               "generator","result","algorithm","subtype","seed1","seed2"]

Configuración de matplotlib

In [ ]:
def plt_setlegend(ax, legend=None, loc='best'):
    if legend is not None:
        l = ax.legend(legend, loc=loc, frameon=True, borderpad=0.5)
    else:
        l = ax.legend(loc=loc, frameon=True, borderpad=0.5)
    l.get_frame().set_linewidth(1)

### Leemos los benchmarks
Separamos los de time de los de result, ya que los últimos se corren una sola vez y pueden tener mucho ruido

In [ ]:
def readDf(wildcard):
    files = glob.glob(os.path.join(BENCH_DIR, wildcard))
    dfs = (pd.read_csv(f, names=CSV_COLUMNS, header=None) for f in files)
    df = pd.concat(dfs, ignore_index=True)
    df["time"] = df["time"] / 1e9
    df["nodes"] = df["gyms"] + df["stops"]
    df["n^2"] = df["nodes"]**2
    df["n^3"] = df["nodes"]**3
    df["n^4"] = df["nodes"]**4
    df["n^5"] = df["nodes"]**5
    df["n^6"] = df["nodes"]**6
    return df

df_time = readDf("*-time-*")
df_time_moch = readDf("*-timemoch-*")
df_result = readDf("*-result-*")
df_grasp_var = readDf("grasp-variables")

### Datos de las mediciones

In [ ]:
df_time[["algorithm","subtype"]].drop_duplicates()

In [ ]:
df_time.describe()

In [ ]:
df_result.describe()

## Graficos de tiempo

In [ ]:
t_is_exacto = df_time["algorithm"] == 'exacto'
t_is_greedy = df_time["algorithm"] == 'greedy'
t_is_local = df_time["algorithm"] == 'local'
t_is_grasp = df_time["algorithm"] == 'grasp'

t_is_bruteforce = df_time["subtype"] == 'none'
t_is_backtracking = df_time["subtype"] == 'backtracking'
t_is_closest = df_time["subtype"] == 'closest'
t_is_furthest = df_time["subtype"] == 'furthest'
t_is_2opt = df_time["subtype"] == 'dos_opt'
t_is_swap = df_time["subtype"] == 'swap'
t_is_swap_min = df_time["subtype"] == 'swap_min'


t_is_not_inf = df_time["result"] != np.inf

tm_is_exacto = df_time_moch["algorithm"] == 'exacto'
tm_is_greedy = df_time_moch["algorithm"] == 'greedy'
tm_is_local = df_time_moch["algorithm"] == 'local'
tm_is_grasp = df_time_moch["algorithm"] == 'grasp'

tm_is_bruteforce = df_time_moch["subtype"] == 'none'
tm_is_backtracking = df_time_moch["subtype"] == 'backtracking'
tm_is_closest = df_time_moch["subtype"] == 'closest'
tm_is_furthest = df_time_moch["subtype"] == 'furthest'
tm_is_2opt = df_time_moch["subtype"] == 'dos_opt'
tm_is_swap = df_time_moch["subtype"] == 'swap'

tm_is_not_inf = df_time_moch["result"] != np.inf

### Algoritmos exactos

In [ ]:
plt.clf()
axis1 = plt.subplot(1,2,1)
plt.ylabel('Tiempo en segundos')
axis2 = plt.subplot(1,2,2)
plt.ylabel('Tiempo en segundos')

df_time[t_is_exacto & t_is_bruteforce].plot(
    x="nodes", y="time", ax=axis1,
    label="fuerza bruta", style='o')

df_time[t_is_exacto & t_is_backtracking].plot(
    x="nodes", y="time", ax=axis2,
    label="backtracking", style='go')

# Seteamos la misma escala para ambos
axis1.set_yscale('log')
axis2.set_yscale('log')
_,_,y1l,y1h = axis1.axis()
_,_,y2l,y2h = axis2.axis()
#axis1.set_ylim(min(y1l,y2l),max(y1h,y2h))
axis2.set_ylim(min(y1l,y2l),max(y1h,y2h))

plt.suptitle("Runtime de los algoritmos exactos, con mochilas 'infinitas'")
plt_setlegend(axis1)
plt_setlegend(axis2)
plt.savefig(GRAPH_DIR + 'time-exacto.png')
plt.show()

In [ ]:
df_time_moch[tm_is_exacto & tm_is_backtracking]["time"].describe()

In [ ]:
plt.clf()
axis1 = plt.subplot(1,2,1)
plt.ylabel('Tiempo en segundos')
axis2 = plt.subplot(1,2,2)
plt.ylabel('Tiempo en segundos')

df_time_moch[tm_is_exacto & tm_is_bruteforce].plot(
    x="bagSize", y="time", ax=axis1,
    label="fuerza bruta", style='o')

df_time_moch[tm_is_exacto & tm_is_backtracking].plot(
    x="bagSize", y="time", ax=axis2,
    label="backtracking", style='go')

axis1.set_xlabel("Tamaño de la mochila")
axis2.set_xlabel("Tamaño de la mochila")

# Seteamos la misma escala para ambos
_,_,y1l,y1h = axis1.axis()
_,_,y2l,y2h = axis2.axis()
axis1.set_ylim(0,max(y1h,y2h))
axis2.set_ylim(0,max(y1h,y2h))

plt.suptitle("Runtime de los algoritmos exactos con 5 gimnasios y 5 paradas")
plt_setlegend(axis1)
plt_setlegend(axis2)
plt.savefig(GRAPH_DIR + 'time-exacto-moch.png')
plt.show()

### Heurísticas greedy

In [ ]:
plt.clf()
axis1 = plt.subplot(1,2,1)
plt.ylabel('Tiempo en segundos')
axis2 = plt.subplot(1,2,2)
plt.ylabel('Tiempo en segundos')

df_time[t_is_greedy & t_is_furthest & t_is_not_inf].plot(
    x="nodes", y="time", ax=axis1,
    label="heurística greedy, closest first", style='o')
df_time[t_is_greedy & t_is_closest & t_is_not_inf].plot(
    x="nodes", y="time", ax=axis2,
    label="heurística greedy, furthest first", style='go')

# Seteamos la misma escala para ambos
x1,x2,y1,y2 = axis1.axis()
axis2.set_ylim(y1,y2)

plt.suptitle("Runtime de las heurísticas greedy, con mochilas 'infinitas'")
plt_setlegend(axis1)
plt_setlegend(axis2)
plt.savefig(GRAPH_DIR + 'time-greedy.png')
plt.show()

In [ ]:
t_grasp = df_time[t_is_greedy & t_is_not_inf]
plt.clf()
sns.jointplot(t_grasp['n^2'], t_grasp['time'], kind="reg")
plt.savefig(GRAPH_DIR + 'time-greedy-correlation.png')
sns.plt.show()

In [ ]:
plt.clf()
axis1 = plt.subplot(1,2,1)
plt.ylabel('Tiempo en segundos')
axis2 = plt.subplot(1,2,2)
plt.ylabel('Tiempo en segundos')

df_time_moch[tm_is_greedy & tm_is_closest].plot(
    x="bagSize", y="time", ax=axis1,
    label="greedy closest first", style='o')

df_time_moch[tm_is_greedy & tm_is_furthest].plot(
    x="bagSize", y="time", ax=axis2,
    label="greedy furthest first", style='go')

axis1.set_xlabel("Tamaño de la mochila")
axis2.set_xlabel("Tamaño de la mochila")

# Seteamos la misma escala para ambos
_,_,y1l,y1h = axis1.axis()
_,_,y2l,y2h = axis2.axis()
axis1.set_ylim(0,max(y1h,y2h))
axis2.set_ylim(0,max(y1h,y2h))

plt.suptitle("Runtime de las heurísticas greedy con 5 gimnasios y 5 paradas")
plt_setlegend(axis1)
plt_setlegend(axis2)
plt.savefig(GRAPH_DIR + 'time-greedy-moch.png')
plt.show()

In [ ]:
df_time_moch[tm_is_greedy][["time","bagSize"]].corr()

In [ ]:
df_time_moch[tm_is_greedy & tm_is_closest]["time"].describe()

### Heurísticas locales

#### 2opt vs swap

In [ ]:
plt.clf()
axis1 = plt.subplot(1,2,1)
plt.ylabel('Tiempo en segundos')
axis2 = plt.subplot(1,2,2)
plt.ylabel('Tiempo en segundos')

ax = df_time[t_is_local & t_is_swap & t_is_not_inf].plot(
    x="nodes", y="time", ax=axis1,
    label="heurística local, swap", style='o')
ax = df_time[t_is_local & t_is_2opt & t_is_not_inf].plot(
    x="nodes", y="time", ax=axis2,
    label="heurística local, 2opt", style='go')

# Seteamos la misma escala para ambos
x1,x2,y1,y2 = axis1.axis()
axis2.set_ylim(y1,y2)

plt.suptitle("Runtime de las heurísticas locales, con mochilas 'infinitas'")
plt_setlegend(axis1)
plt_setlegend(axis2)
plt.savefig(GRAPH_DIR + 'time-local.png')
plt.show()

In [ ]:
sns.plt.clf()
jp1 = sns.jointplot('n^4', 'time',
              data=df_time[t_is_local & t_is_2opt & t_is_not_inf],
              kind="reg")
jp2 = sns.jointplot('n^4', 'time',
              data=df_time[t_is_local & t_is_swap & t_is_not_inf],
              kind="reg")

f = plt.figure()

#subplots migration
f = plt.figure()
for j in [jp1, jp2]:
    for a in j.fig.axes:
        f._axstack.add(f._make_key(a), a)

#subplots size adjustment
f.axes[0].set_position([0.05, 0.05, 0.4,  0.4])
f.axes[1].set_position([0.05, 0.47, 0.4,  0.05])
f.axes[2].set_position([0.47, 0.05, 0.05, 0.4])

f.axes[3].set_position([0.65, 0.05, 0.4,  0.4])
f.axes[4].set_position([0.65, 0.47, 0.4,  0.05])
f.axes[5].set_position([1.07, 0.05, 0.05, 0.4])

plt.savefig(GRAPH_DIR + 'time-local-correlation.png',
            bbox_inches='tight', pad_inches=0)
sns.plt.show()

In [ ]:
plt.clf()
axis1 = plt.subplot(1,2,1)
plt.ylabel('Tiempo en segundos')
axis2 = plt.subplot(1,2,2)
plt.ylabel('Tiempo en segundos')

df_time_moch[tm_is_local & tm_is_swap].plot(
    x="bagSize", y="time", ax=axis1,
    label="local swap", style='o')

df_time_moch[tm_is_local & tm_is_2opt].plot(
    x="bagSize", y="time", ax=axis2,
    label="local 2opt", style='go')

axis1.set_xlabel("Tamaño de la mochila")
axis2.set_xlabel("Tamaño de la mochila")

# Seteamos la misma escala para ambos
_,_,y1l,y1h = axis1.axis()
_,_,y2l,y2h = axis2.axis()
axis1.set_ylim(0,max(y1h,y2h))
axis2.set_ylim(0,max(y1h,y2h))

plt.suptitle("Runtime de las heurísticas locales con 5 gimnasios y 5 paradas")
plt_setlegend(axis1)
plt_setlegend(axis2)
plt.savefig(GRAPH_DIR + 'time-local-moch.png')
plt.show()

In [ ]:
df_time_moch[tm_is_local & tm_is_swap][["time","bagSize"]].corr()

In [ ]:
df_time_moch[tm_is_local & tm_is_2opt][["time","bagSize"]].corr()

In [ ]:
df_time[t_is_local & t_is_swap].describe()

In [ ]:
df_time[t_is_local & t_is_2opt].describe()

In [ ]:
df_time_moch[tm_is_local & tm_is_2opt & tm_is_not_inf]["time"].describe()

#### swap_min vs swap

In [ ]:
df_time[t_is_local & t_is_swap_min & t_is_not_inf].describe()

In [ ]:
plt.clf()
fig, ax = plt.subplots()
plt.ylabel('Tiempo en segundos')

ax = df_time[t_is_local & t_is_swap_min & t_is_not_inf].plot(
    x="nodes", y="time", ax=ax,
    label="swap_min", style='go')

plt.title("Runtime de las dos búsquedas por swap, con mochilas 'infinitas'")
plt_setlegend(ax)
plt.savefig(GRAPH_DIR + 'time-local-swap_min.png')
plt.show()

In [ ]:
plt.clf()
sns.jointplot('n^5', 'time',
              data=df_time[t_is_local & t_is_swap_min & t_is_not_inf],
              kind="reg")
plt.savefig(GRAPH_DIR + 'time-local-swap_min-correlation.png')
sns.plt.show()

### GRASP

In [ ]:
plt.clf()
fig, ax = plt.subplots()

ax = df_time[t_is_grasp & t_is_not_inf].plot(x="nodes", y="time", ax=ax,
                            label="heurística grasp", style='o')
plt.ylabel('Tiempo en segundos')
plt.title("Runtime de la metaheurística GRASP, con mochilas 'infinitas'")
plt_setlegend(ax)
plt.savefig(GRAPH_DIR + 'time-grasp.png')
plt.show()

In [ ]:
t_grasp = df_time[t_is_grasp & t_is_not_inf]
plt.clf()
sns.jointplot(t_grasp['n^5'], t_grasp['time'], kind="reg")
plt.savefig(GRAPH_DIR + 'time-grasp-correlation.png')
sns.plt.show()

In [ ]:
df_time_moch[tm_is_grasp].describe()

In [ ]:
plt.clf()
fig, ax = plt.subplots()

df_time_moch[tm_is_grasp].plot(
    x="bagSize", y="time", ax=ax,
    label="grasp", style='o')

ax.set_xlabel("Tamaño de la mochila")

plt.title("Runtime de la metaheurística GRASP con 20 gimnasios y 20 paradas")
plt_setlegend(ax)
plt.savefig(GRAPH_DIR + 'time-grasp-moch.png')
plt.show()

In [ ]:
df_time_moch[tm_is_grasp][["time","bagSize"]].corr()

In [ ]:
df_time_moch[tm_is_grasp]["time"].describe()

## Graficos de precisión

#### Cosas utiles

In [ ]:
class LogFormatterTeXExponent(pylab.LogFormatter, object):
    """Extends pylab.LogFormatter to use 
    tex notation for tick labels."""
    
    def __init__(self, *args, **kwargs):
        super(LogFormatterTeXExponent, 
              self).__init__(*args, **kwargs)
        
    def __call__(self, *args, **kwargs):
        """Wrap call to parent class with 
        change to tex notation."""
        label = super(LogFormatterTeXExponent, 
                      self).__call__(*args, **kwargs)
        label = re.sub(r'e(\S)0?(\d+)', 
                       r'\\times aoe 10^{\1\2}', 
                       str(label))
        label = "$" + label + "$"
        return label
    
def setLogY(ax):
    plt.yscale('log')
    ax.yaxis.set_minor_formatter(
        LogFormatterTeXExponent(labelOnlyBase=False))
    

### En funcion del exacto

In [ ]:
is_exacto = df_result["algorithm"] == 'exacto'
is_greedy = df_result["algorithm"] == 'greedy'
is_local = df_result["algorithm"] == 'local'
is_grasp = df_result["algorithm"] == 'grasp'

is_closest = df_result["subtype"] == 'closest'
is_furthest = df_result["subtype"] == 'furthest'
is_2opt = df_result["subtype"] == 'dos_opt'
is_swap = df_result["subtype"] == 'swap'

merge_cols = ["gyms","stops","nodes","bagSize","generator","seed1","seed2"]
columns = merge_cols + ["result"]
ext_columns = columns + ["algorithm", "subtype"]

small_exacto = df_result[is_exacto][columns]
small_exacto = small_exacto[small_exacto["result"] != np.inf]
small_exacto.rename(columns={"result": "result_exacto"}, inplace=True)

def getPrecision(data_filter, name):
    tmp = pd.merge(df_result[data_filter][ext_columns],
                           small_exacto, on=merge_cols)
    tmp = tmp[tmp["result"] != np.inf]
    tmp["precision"] = tmp["result"] / tmp["result_exacto"]
    tmp["name"] = name
    
    # Agregamos el tamaño relativo de la medida
    def tamRelativo(row, string=True):
        s = row['stops']
        t = row['bagSize']
        if not string:
            return t/s
        if t == s * 3:
            return 'mochila * 3'
        if t == s * 2:
            return 'mochila * 2'
        if t == s:
            return 'mochila'
        if t == s/2:
            return 'mochila / 2'
        if t == 3:
            return 'size 3'
        return ''
    tmp["sizeBag relativo num"] = tmp.apply(
        lambda row: tamRelativo(row, string=False), axis=1)
    tmp["sizeBag relativo"] = tmp.apply(tamRelativo, axis=1)
    tmp.sort_values(['generator','sizeBag relativo num'], inplace=True)
    return tmp

prec_greedy_closest = getPrecision(is_greedy & is_closest, 'greedy closest first')
prec_greedy_furthest = getPrecision(is_greedy & is_furthest, 'greedy furthest first')
prec_local_2opt = getPrecision(is_local & is_2opt, 'local, 2opt')
prec_local_swap = getPrecision(is_local & is_swap, 'local, swap')
prec_grasp = getPrecision(is_grasp, 'grasp')

prec_all = prec_greedy_closest.append(prec_greedy_furthest
                             ).append(prec_local_2opt).append(prec_local_swap
                             ).append(prec_grasp)

prec_is_random = prec_all['generator'] == 'random'

In [ ]:
plt.clf()
fig, ax = plt.subplots()
setLogY(ax)

sns.boxplot(x='name', y='precision',
            data=prec_all[prec_is_random])
plt.savefig(GRAPH_DIR + 'precision-small-all.png')
plt.show()

In [ ]:
plt.clf()
fig, ax = plt.subplots()
setLogY(ax)

sns.boxplot(x='name', y='precision', hue='generator',
            data=prec_all)
plt.savefig(GRAPH_DIR + 'precision-small-all-bygen.png')
plt.show()

In [ ]:
plt.clf()
fig, ax = plt.subplots()
setLogY(ax)

sns.boxplot(x='name', y='precision', hue='sizeBag relativo', data=prec_all)
plt.savefig(GRAPH_DIR + 'precision-small-all-bysize.png')
plt.show()

In [ ]:
prec_grasp[prec_grasp['generator'] == 'random']["precision"].describe()

In [ ]:
len(prec_grasp[prec_grasp["precision"] == 1]) / len(prec_grasp)

### En funcion del mínimo entre las herísticas

In [ ]:
merge_cols = ["gyms","stops","nodes","bagSize","generator","seed1","seed2"]
columns = merge_cols + ["result"]
min_columns = merge_cols + ["min_result"]
ext_columns = columns + ["algorithm", "subtype"]

minResult = df_result[is_grasp].rename(columns={"result": "min_result"})
minResult = minResult[min_columns]
minResult = minResult[minResult["min_result"] != np.inf]

def mergeMinimums(data_filter):
    global minResult
    minResult = pd.merge(df_result[data_filter][columns], minResult, on=merge_cols)
    minResult['min_result'] = minResult.loc[:, ['result', 'min_result']].min(axis=1)
    minResult = minResult[min_columns]

mergeMinimums(is_greedy & is_closest)
mergeMinimums(is_greedy & is_furthest)
mergeMinimums(is_local & is_2opt)
mergeMinimums(is_local & is_swap)

def getPrecision(data_filter, name):
    tmp = pd.merge(df_result[data_filter][ext_columns], minResult, on=merge_cols)
    tmp = tmp[tmp["result"] != np.inf]
    tmp["precision"] = tmp["result"] / tmp["min_result"]
    tmp["name"] = name
    
    # Agregamos el tamaño relativo de la medida
    def tamRelativo(row, string=True):
        s = row['stops']
        t = row['bagSize']
        if not string:
            return t/s
        if t == s * 3:
            return 'mochila * 3'
        if t == s * 2:
            return 'mochila * 2'
        if t == s:
            return 'mochila'
        if t == s/2:
            return 'mochila / 2'
        if t == 3:
            return 'size 3'
        return ''
    tmp["sizeBag relativo num"] = tmp.apply(
        lambda row: tamRelativo(row, string=False), axis=1)
    tmp["sizeBag relativo"] = tmp.apply(tamRelativo, axis=1)
    tmp.sort_values(['generator','sizeBag relativo num'], inplace=True)
    
    return tmp

b_prec_greedy_closest = getPrecision(is_greedy & is_closest, 'greedy closest first')
b_prec_greedy_furthest = getPrecision(is_greedy & is_furthest, 'greedy furthest first')
b_prec_local_2opt = getPrecision(is_local & is_2opt, 'local, 2opt')
b_prec_local_swap = getPrecision(is_local & is_swap, 'local, swap')
b_prec_grasp = getPrecision(is_grasp, 'grasp')

b_prec_all = b_prec_greedy_closest.append(b_prec_greedy_furthest
                             ).append(b_prec_local_2opt
                             ).append(b_prec_local_swap
                             ).append(b_prec_grasp)

b_prec_is_random = b_prec_all['generator'] == 'random'

In [ ]:
plt.clf()
fig, ax = plt.subplots()
setLogY(ax)

sns.boxplot(x='name', y='precision', data=b_prec_all[b_prec_is_random], ax=ax)
plt.savefig(GRAPH_DIR + 'precision-big-all.png')
plt.show()

In [ ]:
plt.clf()
fig, ax = plt.subplots()
setLogY(ax)

sns.boxplot(x='name', y='precision', hue='generator', data=b_prec_all)
plt.savefig(GRAPH_DIR + 'precision-big-all-bygen.png')
plt.show()

In [ ]:
plt.clf()
fig, ax = plt.subplots()
setLogY(ax)

sns.boxplot(x='name', y='precision', hue='sizeBag relativo', data=b_prec_all)
plt.savefig(GRAPH_DIR + 'precision-big-all-bysize.png')
plt.show()

In [ ]:
b_prec_grasp["precision"].describe()

In [ ]:
len(b_prec_grasp[b_prec_grasp["precision"] == 1]) / len(b_prec_grasp)

In [ ]:
b_prec_local_2opt["precision"].describe()

In [ ]:
len(b_prec_local_2opt[b_prec_local_2opt["precision"] == 1]) / len(b_prec_local_2opt)

## Variables del grasp

In [ ]:
df_grasp_var['limite'] = df_grasp_var.apply(lambda row: float(row['subtype'].split()[0]), axis=1)
df_grasp_var['inicios'] = df_grasp_var.apply(lambda row: float(row['subtype'].split()[1]), axis=1)

# Extraido previamente correndo
# df_grasp_var[['seed1','seed2','result']].groupby(['seed1','seed2']).min()
results = {123: 370836.563283, 654: 338341.100322, 789: 368630.367911}
df_grasp_var['min_result'] = df_grasp_var.apply(lambda row: results[row['seed1']], axis=1)
df_grasp_var['precision'] = df_grasp_var['result'] / df_grasp_var['min_result']

df_gv = df_grasp_var[(df_grasp_var['limite'] < 1.75) & (df_grasp_var['inicios'] < 1.75) &
                     (df_grasp_var['limite'] > 0.15) & (df_grasp_var['inicios'] > 0.15)]
df_gv_slim = df_gv[['limite','inicios','time', 'precision']]

tmp = df_gv_slim.groupby(['limite','inicios']).mean()
tmp = pd.DataFrame(tmp.to_records())
df_gv_2d_li = tmp.pivot(index='limite', columns='inicios', values='time')
df_gv_2d_il = tmp.pivot(index='inicios', columns='limite', values='time')

df_gvr_2d_li = tmp.pivot(index='limite', columns='inicios', values='precision')
df_gvr_2d_il = tmp.pivot(index='inicios', columns='limite', values='precision')

In [ ]:
plt.clf()

cbar_kws = { 'ticks' : [0.01, 0.1, 1, 10] }
ax = sns.heatmap(df_gv_2d_li.iloc[::-1], norm=LogNorm(vmin=0.01, vmax=15), square=True,
                 cbar_kws=cbar_kws)
plt.savefig(GRAPH_DIR + 'grasp-variables-heatmap.png')
plt.show()

In [ ]:
plt.clf()
axis1 = plt.subplot(1,2,1)
plt.yscale('log')
plt.ylabel('Tiempo en segundos')
axis2 = plt.subplot(1,2,2)
plt.yscale('log')

df_gv_2d_li.plot(ax=axis1)
df_gv_2d_il.plot(ax=axis2)

handles, labels = axis1.get_legend_handles_labels()
axis1.legend(handles[::-1], labels[::-1], title='inicios', loc='upper left',
          bbox_to_anchor=(1,1))
handles, labels = axis2.get_legend_handles_labels()
axis2.legend(handles[::-1], labels[::-1], title='limite', loc='upper left',
          bbox_to_anchor=(1,1))

plt.subplots_adjust(wspace=0.5, hspace=None)

plt.savefig(GRAPH_DIR + 'grasp-variables-time.png')
plt.show()

In [ ]:
plt.clf()

ax = sns.heatmap(df_gvr_2d_li.iloc[::-1])
plt.savefig(GRAPH_DIR + 'grasp-variables-heatmap-prec.png')
plt.show()

In [ ]:
tmp = df_gv_slim
tmp = tmp[tmp['inicios'] == tmp['limite']]
tmp[(tmp['inicios'] == 0.4) | (tmp['inicios'] == 1)].groupby(['inicios','limite']).mean()

In [ ]:
plt.clf()
axis1 = plt.subplot(1,2,1)
plt.ylabel('Precision')
axis2 = plt.subplot(1,2,2)

df_gvr_2d_li.plot(ax=axis1)
df_gvr_2d_il.plot(ax=axis2)

axis1.legend(title='inicios', loc='upper left', bbox_to_anchor=(1,1))
axis2.legend(title='limite', loc='upper left', bbox_to_anchor=(1,1))

plt.subplots_adjust(wspace=0.5, hspace=None)

plt.savefig(GRAPH_DIR + 'grasp-variables-prec.png')
plt.show()